In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings

In [3]:
warnings.filterwarnings('ignore', message='Workbook contains no default style')

In [19]:
df = pd.read_csv(r"C:\Users\user\Documents\서울시_포폴\서울시분기별매출\seoul_sales_2023.csv",
                 encoding="cp949")

In [22]:
df.head(2)

,기준_년분기_코드,행정동_코드,행정동_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,월요일_매출_금액,...,시간대_건수~21_매출_건수,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수
0,20231,11110515,청운효자동,CS100001,한식음식점,3251331172,99863,2353732919,897598253,403258206,...,27742,2857,48192,40639,519,12998,19526,18614,20331,16828
1,20231,11110515,청운효자동,CS100002,중식음식점,296642191,8631,191061126,105581065,39842021,...,2906,5,4691,3186,37,1307,2101,1622,1383,1428


In [23]:
df.columns

Index(['기준_년분기_코드', '행정동_코드', '행정동_코드_명', '서비스_업종_코드', '서비스_업종_코드_명',
       '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액',
       '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액',
       '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
       '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수',
       '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수',
       '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수', '시간대_건수~06_매출_건수',
       '시간대_건수~11_매출_건수', '시간대_건수~14_매출_건수', '시간대_건수~17_매출_건수',
       '시간대_건수~21_매출_건수', '시간대_건수~24_매출_건수', '남성_매출_건수', '여성_매출_건수',
       '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수',
       '연령대_50_매출_건수', '연령대_60_이상_매출_건수'],
      dtype='object')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68643 entries, 0 to 68642
Data columns (total 53 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   기준_년분기_코드        68643 non-null  int64 
 1   행정동_코드           68643 non-null  int64 
 2   행정동_코드_명         68643 non-null  object
 3   서비스_업종_코드        68643 non-null  object
 4   서비스_업종_코드_명      68643 non-null  object
 5   당월_매출_금액         68643 non-null  int64 
 6   당월_매출_건수         68643 non-null  int64 
 7   주중_매출_금액         68643 non-null  int64 
 8   주말_매출_금액         68643 non-null  int64 
 9   월요일_매출_금액        68643 non-null  int64 
 10  화요일_매출_금액        68643 non-null  int64 
 11  수요일_매출_금액        68643 non-null  int64 
 12  목요일_매출_금액        68643 non-null  int64 
 13  금요일_매출_금액        68643 non-null  int64 
 14  토요일_매출_금액        68643 non-null  int64 
 15  일요일_매출_금액        68643 non-null  int64 
 16  시간대_00~06_매출_금액  68643 non-null  int64 
 17  시간대_06~11_매출_금액  68643 non-null

In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import pymysql

# ===== 설정 =====
ROOT_DIR = r"C:\Users\user\Documents\서울시_포폴\서울시분기별매출"
TABLE = "stg_seoul_sales_quarter"

MYSQL = dict(
    host="localhost",
    user="root",
    password="********",
    db="seoul",
    port=3306,
    charset="utf8mb4",
    autocommit=False
)

CHUNK_SIZE = 20000

# ===== 유틸 =====
def read_csv_auto(path):
    for enc in ("utf-8-sig", "cp949", "euc-kr"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    raise ValueError(f"인코딩 실패: {path}")

def normalize_cols(df):
    # 컬럼명 좌우 공백 제거
    df.columns = df.columns.astype(str).str.strip()

    # ~ 를 _ 로 통일 (시간대_00~06 → 시간대_00_06)
    rename = {}
    for c in df.columns:
        c2 = c.replace("~", "_")
        # 건수 컬럼이 '시간대_건수~06_매출_건수'처럼 이상하게 들어온 경우가 있어 보여서 정리
        c2 = c2.replace("시간대_건수_", "시간대_")
        rename[c] = c2
    return df.rename(columns=rename)

def to_int_cols(df, exclude):
    df = df.copy()
    for c in df.columns:
        if c in exclude:
            continue
        # 숫자 문자열에 콤마가 있을 가능성 대비
        df[c] = (df[c].astype(str)
                 .str.replace(",", "", regex=False)
                 .str.strip())
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def get_insert_sql(cols):
    col_sql = ",".join([f"`{c}`" for c in cols])
    placeholders = ",".join(["%s"] * len(cols))
    return f"INSERT IGNORE INTO {TABLE} ({col_sql}) VALUES ({placeholders})"

def load_sales_files():
    conn = pymysql.connect(**MYSQL)

    files = sorted(Path(ROOT_DIR).glob("seoul_sales_*.csv"))
    if not files:
        raise ValueError("seoul_sales_*.csv 파일을 못 찾았어. 경로 확인해줘.")

    # DB 컬럼 순서 (DDL 기준)
    cols = [
        "기준_년분기_코드","행정동_코드","행정동_코드_명","서비스_업종_코드","서비스_업종_코드_명",
        "당월_매출_금액","당월_매출_건수",
        "주중_매출_금액","주말_매출_금액","월요일_매출_금액","화요일_매출_금액","수요일_매출_금액",
        "목요일_매출_금액","금요일_매출_금액","토요일_매출_금액","일요일_매출_금액",
        "시간대_00_06_매출_금액","시간대_06_11_매출_금액","시간대_11_14_매출_금액",
        "시간대_14_17_매출_금액","시간대_17_21_매출_금액","시간대_21_24_매출_금액",
        "남성_매출_금액","여성_매출_금액",
        "연령대_10_매출_금액","연령대_20_매출_금액","연령대_30_매출_금액","연령대_40_매출_금액",
        "연령대_50_매출_금액","연령대_60_이상_매출_금액",
        "주중_매출_건수","주말_매출_건수","월요일_매출_건수","화요일_매출_건수","수요일_매출_건수",
        "목요일_매출_건수","금요일_매출_건수","토요일_매출_건수","일요일_매출_건수",
        "시간대_00_06_매출_건수","시간대_06_11_매출_건수","시간대_11_14_매출_건수",
        "시간대_14_17_매출_건수","시간대_17_21_매출_건수","시간대_21_24_매출_건수",
        "남성_매출_건수","여성_매출_건수",
        "연령대_10_매출_건수","연령대_20_매출_건수","연령대_30_매출_건수","연령대_40_매출_건수",
        "연령대_50_매출_건수","연령대_60_이상_매출_건수",
        "source_file"
    ]

    sql = get_insert_sql(cols)

    ok, fail = 0, 0
    with conn.cursor() as cur:
        for f in files:
            path = str(f)
            print(f"\n[START] {f.name}")

            try:
                df = read_csv_auto(path)
                df = normalize_cols(df)

                # 필수 컬럼 체크
                required = {"기준_년분기_코드","행정동_코드","서비스_업종_코드"}
                missing = required - set(df.columns)
                if missing:
                    raise ValueError(f"필수 컬럼 누락: {missing}")

                # 문자열 컬럼 제외하고 숫자 변환
                exclude = {"기준_년분기_코드","행정동_코드_명","서비스_업종_코드","서비스_업종_코드_명"}
                df = to_int_cols(df, exclude=exclude)

                # 행정동_코드가 숫자인지(문자면 숫자화)
                df["행정동_코드"] = pd.to_numeric(df["행정동_코드"], errors="coerce")
                df = df.dropna(subset=["기준_년분기_코드","행정동_코드","서비스_업종_코드"])

                df["행정동_코드"] = df["행정동_코드"].astype(int)

                df["source_file"] = f.name

                # NaN -> None
                sub = df[cols].astype(object).where(pd.notnull(df[cols]), None)
                data = [tuple(r) for r in sub.to_numpy()]

                for i in range(0, len(data), CHUNK_SIZE):
                    cur.executemany(sql, data[i:i+CHUNK_SIZE])
                    print(f"  inserted {min(i+CHUNK_SIZE, len(data))}/{len(data)}")

                conn.commit()
                ok += 1
                print(f"[DONE] {f.name} rows={len(df)}")

            except Exception as e:
                conn.rollback()
                fail += 1
                print(f"[FAIL] {f.name} -> {e}")

    conn.close()
    print("\n===================================")
    print(f"files ok={ok}, fail={fail}")
    print("ALL DONE")

# 실행
load_sales_files()



[START] seoul_sales_2023.csv
[FAIL] seoul_sales_2023.csv -> "['시간대_00_06_매출_건수', '시간대_06_11_매출_건수', '시간대_11_14_매출_건수', '시간대_14_17_매출_건수', '시간대_17_21_매출_건수', '시간대_21_24_매출_건수'] not in index"

[START] seoul_sales_2024.csv
[FAIL] seoul_sales_2024.csv -> "['시간대_00_06_매출_건수', '시간대_06_11_매출_건수', '시간대_11_14_매출_건수', '시간대_14_17_매출_건수', '시간대_17_21_매출_건수', '시간대_21_24_매출_건수'] not in index"

[START] seoul_sales_20251.csv
[FAIL] seoul_sales_20251.csv -> 필수 컬럼 누락: {'행정동_코드', '기준_년분기_코드', '서비스_업종_코드'}

[START] seoul_sales_20252.csv
[FAIL] seoul_sales_20252.csv -> 필수 컬럼 누락: {'행정동_코드', '기준_년분기_코드', '서비스_업종_코드'}

files ok=0, fail=4
ALL DONE


## DB 적재 실패 후 전처리 

### 2023, 2024년도 컬럼에 틸트 들어감. '~' 제거하기

In [25]:
import re

def normalize_cols(df):
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip()

    rename = {}
    for c in df.columns:
        c2 = c

        # 1) ~ 를 _ 로
        c2 = c2.replace("~", "_")

        # 2) '시간대_00_06_매출_금액' 같은 형태 만들기 (금액)
        c2 = re.sub(r"^시간대_(\d{2})_(\d{2})_매출_금액$", r"시간대_\1_\2_매출_금액", c2)

        # 3) 문제의 '시간대_건수_06_매출_건수' -> '시간대_00_06_매출_건수' 로 변환
        # 원본: 시간대_건수~06_매출_건수 => 시간대_건수_06_매출_건수 로 바뀐 상태이므로 이걸 잡는다
        m = re.match(r"^시간대_건수_(\d{2})_매출_건수$", c2)
        if m:
            end = m.group(1)
            # end에 따라 start 결정
            start_map = {"06": "00", "11": "06", "14": "11", "17": "14", "21": "17", "24": "21"}
            start = start_map.get(end)
            if start:
                c2 = f"시간대_{start}_{end}_매출_건수"

        rename[c] = c2

    df = df.rename(columns=rename)

    # (선택) 중복 컬럼 있으면 바로 체크
    if df.columns.duplicated().any():
        dup = df.columns[df.columns.duplicated()].tolist()
        raise ValueError(f"정규화 후 중복 컬럼 발생: {dup}")

    return df

In [27]:
normalize_cols(df).head(1)

,기준_년분기_코드,행정동_코드,행정동_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,월요일_매출_금액,...,시간대_17_21_매출_건수,시간대_21_24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수
0,20231,11110515,청운효자동,CS100001,한식음식점,3251331172,99863,2353732919,897598253,403258206,...,27742,2857,48192,40639,519,12998,19526,18614,20331,16828


In [54]:
df = normalize_cols(df)

In [55]:
df.head(2)

,기준_년분기_코드,행정동_코드,행정동_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,월요일_매출_금액,...,시간대_17_21_매출_건수,시간대_21_24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수
0,20231,11110515,청운효자동,CS100001,한식음식점,3251331172,99863,2353732919,897598253,403258206,...,27742,2857,48192,40639,519,12998,19526,18614,20331,16828
1,20231,11110515,청운효자동,CS100002,중식음식점,296642191,8631,191061126,105581065,39842021,...,2906,5,4691,3186,37,1307,2101,1622,1383,1428


### 2025년 전처리

In [29]:
df_2025 = pd.read_csv(r"C:\Users\user\Documents\서울시_포폴\서울시분기별매출\seoul_sales_20251.csv",
                 encoding="utf-8")

In [30]:
df_2025

,STDR_YYQU_CD,ADSTRD_CD,ADSTRD_CD_NM,SVC_INDUTY_CD,SVC_INDUTY_CD_NM,THSMON_SELNG_AMT,THSMON_SELNG_CO,MDWK_SELNG_AMT,WKEND_SELNG_AMT,MON_SELNG_AMT,...,TMZON_17_21_SELNG_CO,TMZON_21_24_SELNG_CO,ML_SELNG_CO,FML_SELNG_CO,AGRDE_10_SELNG_CO,AGRDE_20_SELNG_CO,AGRDE_30_SELNG_CO,AGRDE_40_SELNG_CO,AGRDE_50_SELNG_CO,AGRDE_60_ABOVE_SELNG_CO
0,20251,11110515,청운효자동,CS100001,한식음식점,3383806701,96387,2431057236,952749465,499163632,...,24404,3374,45869,39826,362,12632,20287,17160,18600,16650
1,20251,11110515,청운효자동,CS100003,일식음식점,215368672,11058,147162042,68206630,11798348,...,2310,0,4564,5847,181,2650,2990,1931,1821,841
2,20251,11110515,청운효자동,CS100004,양식음식점,2217678995,40996,1366517021,851161974,146011020,...,13579,1883,12119,26456,201,10241,12623,6422,6134,2948
3,20251,11110515,청운효자동,CS100005,제과점,451701842,31963,280525189,171176653,46513907,...,8542,403,10977,19863,158,5513,7854,6365,6144,4804
4,20251,11110515,청운효자동,CS100007,치킨전문점,109093937,3096,69956483,39137454,4686215,...,2150,564,1530,1405,28,405,699,534,755,515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16765,20251,11740700,둔촌2동,CS300031,가구,111456513,97,88000746,23455767,25666306,...,28,0,46,37,0,0,8,14,23,38
16766,20251,11740700,둔촌2동,CS300033,철물점,305930175,1991,238109507,67820668,28780268,...,356,0,1245,618,0,42,314,316,509,683
16767,20251,11740700,둔촌2동,CS300035,인테리어,327607797,7370,273446831,54160966,108530511,...,417,0,5980,695,0,417,556,834,973,3894
16768,20251,11740700,둔촌2동,CS300036,조명용품,3844985,373,2349181,1495804,527368,...,86,0,249,114,0,0,29,96,48,191


In [31]:
df_2025.columns

Index(['STDR_YYQU_CD', 'ADSTRD_CD', 'ADSTRD_CD_NM', 'SVC_INDUTY_CD',
       'SVC_INDUTY_CD_NM', 'THSMON_SELNG_AMT', 'THSMON_SELNG_CO',
       'MDWK_SELNG_AMT', 'WKEND_SELNG_AMT', 'MON_SELNG_AMT', 'TUES_SELNG_AMT',
       'WED_SELNG_AMT', 'THUR_SELNG_AMT', 'FRI_SELNG_AMT', 'SAT_SELNG_AMT',
       'SUN_SELNG_AMT', 'TMZON_00_06_SELNG_AMT', 'TMZON_06_11_SELNG_AMT',
       'TMZON_11_14_SELNG_AMT', 'TMZON_14_17_SELNG_AMT',
       'TMZON_17_21_SELNG_AMT', 'TMZON_21_24_SELNG_AMT', 'ML_SELNG_AMT',
       'FML_SELNG_AMT', 'AGRDE_10_SELNG_AMT', 'AGRDE_20_SELNG_AMT',
       'AGRDE_30_SELNG_AMT', 'AGRDE_40_SELNG_AMT', 'AGRDE_50_SELNG_AMT',
       'AGRDE_60_ABOVE_SELNG_AMT', 'MDWK_SELNG_CO', 'WKEND_SELNG_CO',
       'MON_SELNG_CO', 'TUES_SELNG_CO', 'WED_SELNG_CO', 'THUR_SELNG_CO',
       'FRI_SELNG_CO', 'SAT_SELNG_CO', 'SUN_SELNG_CO', 'TMZON_00_06_SELNG_CO',
       'TMZON_06_11_SELNG_CO', 'TMZON_11_14_SELNG_CO', 'TMZON_14_17_SELNG_CO',
       'TMZON_17_21_SELNG_CO', 'TMZON_21_24_SELNG_CO', 'ML_SELNG

In [32]:
df_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16770 entries, 0 to 16769
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   STDR_YYQU_CD              16770 non-null  int64 
 1   ADSTRD_CD                 16770 non-null  int64 
 2   ADSTRD_CD_NM              16770 non-null  object
 3   SVC_INDUTY_CD             16770 non-null  object
 4   SVC_INDUTY_CD_NM          16770 non-null  object
 5   THSMON_SELNG_AMT          16770 non-null  int64 
 6   THSMON_SELNG_CO           16770 non-null  int64 
 7   MDWK_SELNG_AMT            16770 non-null  int64 
 8   WKEND_SELNG_AMT           16770 non-null  int64 
 9   MON_SELNG_AMT             16770 non-null  int64 
 10  TUES_SELNG_AMT            16770 non-null  int64 
 11  WED_SELNG_AMT             16770 non-null  int64 
 12  THUR_SELNG_AMT            16770 non-null  int64 
 13  FRI_SELNG_AMT             16770 non-null  int64 
 14  SAT_SELNG_AMT         

In [34]:
# 1) 2025 영문 컬럼 → stg 한글 컬럼 매핑

In [33]:
MAP_2025_TO_STG = {
    "STDR_YYQU_CD": "기준_년분기_코드",
    "ADSTRD_CD": "행정동_코드",
    "ADSTRD_CD_NM": "행정동_코드_명",
    "SVC_INDUTY_CD": "서비스_업종_코드",
    "SVC_INDUTY_CD_NM": "서비스_업종_코드_명",

    "THSMON_SELNG_AMT": "당월_매출_금액",
    "THSMON_SELNG_CO": "당월_매출_건수",

    "MDWK_SELNG_AMT": "주중_매출_금액",
    "WKEND_SELNG_AMT": "주말_매출_금액",
    "MON_SELNG_AMT": "월요일_매출_금액",
    "TUES_SELNG_AMT": "화요일_매출_금액",
    "WED_SELNG_AMT": "수요일_매출_금액",
    "THUR_SELNG_AMT": "목요일_매출_금액",
    "FRI_SELNG_AMT": "금요일_매출_금액",
    "SAT_SELNG_AMT": "토요일_매출_금액",
    "SUN_SELNG_AMT": "일요일_매출_금액",

    "TMZON_00_06_SELNG_AMT": "시간대_00_06_매출_금액",
    "TMZON_06_11_SELNG_AMT": "시간대_06_11_매출_금액",
    "TMZON_11_14_SELNG_AMT": "시간대_11_14_매출_금액",
    "TMZON_14_17_SELNG_AMT": "시간대_14_17_매출_금액",
    "TMZON_17_21_SELNG_AMT": "시간대_17_21_매출_금액",
    "TMZON_21_24_SELNG_AMT": "시간대_21_24_매출_금액",

    "ML_SELNG_AMT": "남성_매출_금액",
    "FML_SELNG_AMT": "여성_매출_금액",
    "AGRDE_10_SELNG_AMT": "연령대_10_매출_금액",
    "AGRDE_20_SELNG_AMT": "연령대_20_매출_금액",
    "AGRDE_30_SELNG_AMT": "연령대_30_매출_금액",
    "AGRDE_40_SELNG_AMT": "연령대_40_매출_금액",
    "AGRDE_50_SELNG_AMT": "연령대_50_매출_금액",
    "AGRDE_60_ABOVE_SELNG_AMT": "연령대_60_이상_매출_금액",

    "MDWK_SELNG_CO": "주중_매출_건수",
    "WKEND_SELNG_CO": "주말_매출_건수",
    "MON_SELNG_CO": "월요일_매출_건수",
    "TUES_SELNG_CO": "화요일_매출_건수",
    "WED_SELNG_CO": "수요일_매출_건수",
    "THUR_SELNG_CO": "목요일_매출_건수",
    "FRI_SELNG_CO": "금요일_매출_건수",
    "SAT_SELNG_CO": "토요일_매출_건수",
    "SUN_SELNG_CO": "일요일_매출_건수",

    "TMZON_00_06_SELNG_CO": "시간대_00_06_매출_건수",
    "TMZON_06_11_SELNG_CO": "시간대_06_11_매출_건수",
    "TMZON_11_14_SELNG_CO": "시간대_11_14_매출_건수",
    "TMZON_14_17_SELNG_CO": "시간대_14_17_매출_건수",
    "TMZON_17_21_SELNG_CO": "시간대_17_21_매출_건수",
    "TMZON_21_24_SELNG_CO": "시간대_21_24_매출_건수",

    "ML_SELNG_CO": "남성_매출_건수",
    "FML_SELNG_CO": "여성_매출_건수",
    "AGRDE_10_SELNG_CO": "연령대_10_매출_건수",
    "AGRDE_20_SELNG_CO": "연령대_20_매출_건수",
    "AGRDE_30_SELNG_CO": "연령대_30_매출_건수",
    "AGRDE_40_SELNG_CO": "연령대_40_매출_건수",
    "AGRDE_50_SELNG_CO": "연령대_50_매출_건수",
    "AGRDE_60_ABOVE_SELNG_CO": "연령대_60_이상_매출_건수",
}

In [35]:
# 2) 2025 파일을 stg 스키마로 “표준화”

In [36]:
def standardize_to_stg(df):
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip()

    # 2025 영문 스키마면 rename
    if "STDR_YYQU_CD" in df.columns and "ADSTRD_CD" in df.columns:
        df = df.rename(columns=MAP_2025_TO_STG)

    # 기존(한글) 스키마면 normalize 처리(틸드 등)만 적용
    # (여기서는 최소만)
    df.columns = df.columns.str.replace("~", "_", regex=False).str.replace(" ", "", regex=False)

    # 타입 정리 (DDL과 맞추기)
    df["기준_년분기_코드"] = df["기준_년분기_코드"].astype(str).str.strip()  # VARCHAR(6)
    df["행정동_코드"] = pd.to_numeric(df["행정동_코드"], errors="coerce")

    return df

In [38]:
standardize_to_stg(df_2025).head(2)

,기준_년분기_코드,행정동_코드,행정동_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,월요일_매출_금액,...,시간대_17_21_매출_건수,시간대_21_24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수
0,20251,11110515,청운효자동,CS100001,한식음식점,3383806701,96387,2431057236,952749465,499163632,...,24404,3374,45869,39826,362,12632,20287,17160,18600,16650
1,20251,11110515,청운효자동,CS100003,일식음식점,215368672,11058,147162042,68206630,11798348,...,2310,0,4564,5847,181,2650,2990,1931,1821,841


In [41]:
df_2025= df_2025.rename(columns=MAP_2025_TO_STG)

In [42]:
df_2025.head(1)

,기준_년분기_코드,행정동_코드,행정동_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,월요일_매출_금액,...,시간대_17_21_매출_건수,시간대_21_24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수
0,20251,11110515,청운효자동,CS100001,한식음식점,3383806701,96387,2431057236,952749465,499163632,...,24404,3374,45869,39826,362,12632,20287,17160,18600,16650


In [ ]:
# 3) “필수 컬럼 누락” 재발 방지 체크

In [43]:
required = {"기준_년분기_코드","행정동_코드","서비스_업종_코드"}
missing = required - set(df_2025.columns)
if missing:
    raise ValueError(f"필수 컬럼 누락(표준화 후에도): {missing}")

In [44]:
df_2025.columns[:10]

Index(['기준_년분기_코드', '행정동_코드', '행정동_코드_명', '서비스_업종_코드', '서비스_업종_코드_명',
       '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액'],
      dtype='object')

In [45]:
# 분기 포맷 통일 (20251 → 2025Q1):

In [47]:
def to_yyyyqn(x):
    s = str(x).strip()
    if len(s) == 5 and s[:4].isdigit() and s[4].isdigit():
        return f"{s[:4]}Q{s[4]}"
    return s

df_2025["기준_년분기_코드"] = df_2025["기준_년분기_코드"].apply(to_yyyyqn)

In [48]:
df_2025.head(2)

,기준_년분기_코드,행정동_코드,행정동_코드_명,서비스_업종_코드,서비스_업종_코드_명,당월_매출_금액,당월_매출_건수,주중_매출_금액,주말_매출_금액,월요일_매출_금액,...,시간대_17_21_매출_건수,시간대_21_24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수
0,2025Q1,11110515,청운효자동,CS100001,한식음식점,3383806701,96387,2431057236,952749465,499163632,...,24404,3374,45869,39826,362,12632,20287,17160,18600,16650
1,2025Q1,11110515,청운효자동,CS100003,일식음식점,215368672,11058,147162042,68206630,11798348,...,2310,0,4564,5847,181,2650,2990,1931,1821,841


### 컬럼 수정 후 비교

In [58]:
# 2023, 2024 컬럼

In [57]:
df.columns

Index(['기준_년분기_코드', '행정동_코드', '행정동_코드_명', '서비스_업종_코드', '서비스_업종_코드_명',
       '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액',
       '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00_06_매출_금액', '시간대_06_11_매출_금액', '시간대_11_14_매출_금액',
       '시간대_14_17_매출_금액', '시간대_17_21_매출_금액', '시간대_21_24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
       '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수',
       '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수',
       '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수', '시간대_00_06_매출_건수',
       '시간대_06_11_매출_건수', '시간대_11_14_매출_건수', '시간대_14_17_매출_건수',
       '시간대_17_21_매출_건수', '시간대_21_24_매출_건수', '남성_매출_건수', '여성_매출_건수',
       '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수',
       '연령대_50_매출_건수', '연령대_60_이상_매출_건수'],
      dtype='object')

In [59]:
# 2025 컬럼

In [52]:
df_2025.columns

Index(['기준_년분기_코드', '행정동_코드', '행정동_코드_명', '서비스_업종_코드', '서비스_업종_코드_명',
       '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액', '월요일_매출_금액',
       '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액',
       '일요일_매출_금액', '시간대_00_06_매출_금액', '시간대_06_11_매출_금액', '시간대_11_14_매출_금액',
       '시간대_14_17_매출_금액', '시간대_17_21_매출_금액', '시간대_21_24_매출_금액', '남성_매출_금액',
       '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액',
       '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '주중_매출_건수',
       '주말_매출_건수', '월요일_매출_건수', '화요일_매출_건수', '수요일_매출_건수', '목요일_매출_건수',
       '금요일_매출_건수', '토요일_매출_건수', '일요일_매출_건수', '시간대_00_06_매출_건수',
       '시간대_06_11_매출_건수', '시간대_11_14_매출_건수', '시간대_14_17_매출_건수',
       '시간대_17_21_매출_건수', '시간대_21_24_매출_건수', '남성_매출_건수', '여성_매출_건수',
       '연령대_10_매출_건수', '연령대_20_매출_건수', '연령대_30_매출_건수', '연령대_40_매출_건수',
       '연령대_50_매출_건수', '연령대_60_이상_매출_건수'],
      dtype='object')

## DB 적재

In [60]:
import re
from pathlib import Path
import pandas as pd
import pymysql

ROOT_DIR = r"C:\Users\user\Documents\서울시_포폴\서울시분기별매출"
TABLE = "stg_seoul_sales_quarter"

MYSQL = dict(
    host="localhost",
    user="root",
    password="Abcd1234!",
    db="seoul",
    port=3306,
    charset="utf8mb4",
    autocommit=False
)

CHUNK_SIZE = 20000

# ========== 2025(영문) -> STG(한글) 매핑 ==========
MAP_2025_TO_STG = {
    "STDR_YYQU_CD": "기준_년분기_코드",
    "ADSTRD_CD": "행정동_코드",
    "ADSTRD_CD_NM": "행정동_코드_명",
    "SVC_INDUTY_CD": "서비스_업종_코드",
    "SVC_INDUTY_CD_NM": "서비스_업종_코드_명",

    "THSMON_SELNG_AMT": "당월_매출_금액",
    "THSMON_SELNG_CO": "당월_매출_건수",

    "MDWK_SELNG_AMT": "주중_매출_금액",
    "WKEND_SELNG_AMT": "주말_매출_금액",
    "MON_SELNG_AMT": "월요일_매출_금액",
    "TUES_SELNG_AMT": "화요일_매출_금액",
    "WED_SELNG_AMT": "수요일_매출_금액",
    "THUR_SELNG_AMT": "목요일_매출_금액",
    "FRI_SELNG_AMT": "금요일_매출_금액",
    "SAT_SELNG_AMT": "토요일_매출_금액",
    "SUN_SELNG_AMT": "일요일_매출_금액",

    "TMZON_00_06_SELNG_AMT": "시간대_00_06_매출_금액",
    "TMZON_06_11_SELNG_AMT": "시간대_06_11_매출_금액",
    "TMZON_11_14_SELNG_AMT": "시간대_11_14_매출_금액",
    "TMZON_14_17_SELNG_AMT": "시간대_14_17_매출_금액",
    "TMZON_17_21_SELNG_AMT": "시간대_17_21_매출_금액",
    "TMZON_21_24_SELNG_AMT": "시간대_21_24_매출_금액",

    "ML_SELNG_AMT": "남성_매출_금액",
    "FML_SELNG_AMT": "여성_매출_금액",
    "AGRDE_10_SELNG_AMT": "연령대_10_매출_금액",
    "AGRDE_20_SELNG_AMT": "연령대_20_매출_금액",
    "AGRDE_30_SELNG_AMT": "연령대_30_매출_금액",
    "AGRDE_40_SELNG_AMT": "연령대_40_매출_금액",
    "AGRDE_50_SELNG_AMT": "연령대_50_매출_금액",
    "AGRDE_60_ABOVE_SELNG_AMT": "연령대_60_이상_매출_금액",

    "MDWK_SELNG_CO": "주중_매출_건수",
    "WKEND_SELNG_CO": "주말_매출_건수",
    "MON_SELNG_CO": "월요일_매출_건수",
    "TUES_SELNG_CO": "화요일_매출_건수",
    "WED_SELNG_CO": "수요일_매출_건수",
    "THUR_SELNG_CO": "목요일_매출_건수",
    "FRI_SELNG_CO": "금요일_매출_건수",
    "SAT_SELNG_CO": "토요일_매출_건수",
    "SUN_SELNG_CO": "일요일_매출_건수",

    "TMZON_00_06_SELNG_CO": "시간대_00_06_매출_건수",
    "TMZON_06_11_SELNG_CO": "시간대_06_11_매출_건수",
    "TMZON_11_14_SELNG_CO": "시간대_11_14_매출_건수",
    "TMZON_14_17_SELNG_CO": "시간대_14_17_매출_건수",
    "TMZON_17_21_SELNG_CO": "시간대_17_21_매출_건수",
    "TMZON_21_24_SELNG_CO": "시간대_21_24_매출_건수",

    "ML_SELNG_CO": "남성_매출_건수",
    "FML_SELNG_CO": "여성_매출_건수",
    "AGRDE_10_SELNG_CO": "연령대_10_매출_건수",
    "AGRDE_20_SELNG_CO": "연령대_20_매출_건수",
    "AGRDE_30_SELNG_CO": "연령대_30_매출_건수",
    "AGRDE_40_SELNG_CO": "연령대_40_매출_건수",
    "AGRDE_50_SELNG_CO": "연령대_50_매출_건수",
    "AGRDE_60_ABOVE_SELNG_CO": "연령대_60_이상_매출_건수",
}

# ========== 유틸 ==========
def read_csv_auto(path: str) -> pd.DataFrame:
    last_err = None
    for enc in ("utf-8-sig", "utf-8", "cp949", "euc-kr"):
        for header in (0, 1, 2):
            try:
                df = pd.read_csv(path, encoding=enc, header=header)
                # 컬럼명이 너무 이상하면(전부 숫자/Unnamed) 헤더로 보기 실패 케이스라 제외
                if df.columns.astype(str).str.contains("Unnamed").mean() > 0.5:
                    continue
                return df
            except Exception as e:
                last_err = e
                continue
    raise ValueError(f"CSV 읽기 실패: {path} / last_err={last_err}")

def to_yyyyqn(x) -> str:
    s = str(x).strip()
    # 20251 -> 2025Q1
    if len(s) == 5 and s[:4].isdigit() and s[4].isdigit():
        return f"{s[:4]}Q{s[4]}"
    # 이미 2023Q1 형태면 그대로
    return s

def normalize_cols_kor(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip()
    rename = {}
    for c in df.columns:
        c2 = c.replace("~", "_").replace(" ", "")

        # 2023/2024에 있었던 '시간대_건수~06_매출_건수' 같은 꼬임 처리
        # ~ -> _ 된 이후를 기준으로 잡음: 시간대_건수_06_매출_건수
        m = re.match(r"^시간대_건수_(\d{2})_매출_건수$", c2)
        if m:
            end = m.group(1)
            start_map = {"06": "00", "11": "06", "14": "11", "17": "14", "21": "17", "24": "21"}
            start = start_map.get(end)
            if start:
                c2 = f"시간대_{start}_{end}_매출_건수"

        rename[c] = c2

    df = df.rename(columns=rename)

    if df.columns.duplicated().any():
        dup = df.columns[df.columns.duplicated()].tolist()
        raise ValueError(f"정규화 후 중복 컬럼 발생: {dup}")

    return df

def standardize_to_stg(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = df.columns.astype(str).str.strip()

    # 2025 영문 스키마면 한글로 변환
    if "STDR_YYQU_CD" in df.columns and "ADSTRD_CD" in df.columns:
        df = df.rename(columns=MAP_2025_TO_STG)

    # 한글 스키마 정규화(틸드, 시간대 건수 꼬임 등)
    df = normalize_cols_kor(df)

    # 분기키 통일: YYYYQn
    df["기준_년분기_코드"] = df["기준_년분기_코드"].apply(to_yyyyqn)

    return df

def to_numeric_cols(df: pd.DataFrame, exclude: set) -> pd.DataFrame:
    df = df.copy()
    for c in df.columns:
        if c in exclude:
            continue
        df[c] = (df[c].astype(str).str.replace(",", "", regex=False).str.strip())
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df

def get_insert_sql(cols):
    col_sql = ",".join([f"`{c}`" for c in cols])
    placeholders = ",".join(["%s"] * len(cols))
    return f"INSERT IGNORE INTO {TABLE} ({col_sql}) VALUES ({placeholders})"

# ========== 적재 ==========
def load_sales_files():
    conn = pymysql.connect(**MYSQL)

    files = sorted(Path(ROOT_DIR).glob("seoul_sales_*.csv"))
    if not files:
        raise ValueError("seoul_sales_*.csv 파일을 못 찾았어. 경로 확인해줘.")

    # DB 컬럼 순서 (DDL 기준) + source_file
    cols = [
        "기준_년분기_코드","행정동_코드","행정동_코드_명","서비스_업종_코드","서비스_업종_코드_명",
        "당월_매출_금액","당월_매출_건수",
        "주중_매출_금액","주말_매출_금액","월요일_매출_금액","화요일_매출_금액","수요일_매출_금액",
        "목요일_매출_금액","금요일_매출_금액","토요일_매출_금액","일요일_매출_금액",
        "시간대_00_06_매출_금액","시간대_06_11_매출_금액","시간대_11_14_매출_금액",
        "시간대_14_17_매출_금액","시간대_17_21_매출_금액","시간대_21_24_매출_금액",
        "남성_매출_금액","여성_매출_금액",
        "연령대_10_매출_금액","연령대_20_매출_금액","연령대_30_매출_금액","연령대_40_매출_금액",
        "연령대_50_매출_금액","연령대_60_이상_매출_금액",
        "주중_매출_건수","주말_매출_건수","월요일_매출_건수","화요일_매출_건수","수요일_매출_건수",
        "목요일_매출_건수","금요일_매출_건수","토요일_매출_건수","일요일_매출_건수",
        "시간대_00_06_매출_건수","시간대_06_11_매출_건수","시간대_11_14_매출_건수",
        "시간대_14_17_매출_건수","시간대_17_21_매출_건수","시간대_21_24_매출_건수",
        "남성_매출_건수","여성_매출_건수",
        "연령대_10_매출_건수","연령대_20_매출_건수","연령대_30_매출_건수","연령대_40_매출_건수",
        "연령대_50_매출_건수","연령대_60_이상_매출_건수",
        "source_file"
    ]

    sql = get_insert_sql(cols)

    ok, fail = 0, 0
    with conn.cursor() as cur:
        for f in files:
            print(f"\n[START] {f.name}")
            try:
                df = read_csv_auto(str(f))
                df = standardize_to_stg(df)

                # 필수 컬럼 체크 (표준화 이후!)
                required = {"기준_년분기_코드","행정동_코드","서비스_업종_코드"}
                missing = required - set(df.columns)
                if missing:
                    raise ValueError(f"필수 컬럼 누락(표준화 후): {missing}")

                # 숫자 변환(문자 컬럼 제외)
                exclude = {"기준_년분기_코드","행정동_코드_명","서비스_업종_코드","서비스_업종_코드_명"}
                df = to_numeric_cols(df, exclude=exclude)

                # key 컬럼 정리
                df["행정동_코드"] = pd.to_numeric(df["행정동_코드"], errors="coerce")
                df = df.dropna(subset=["기준_년분기_코드","행정동_코드","서비스_업종_코드"])
                df["행정동_코드"] = df["행정동_코드"].astype(int)

                # (선택) 서비스 업종 코드 공백 제거
                df["서비스_업종_코드"] = df["서비스_업종_코드"].astype(str).str.strip()

                df["source_file"] = f.name

                # 필요한 컬럼이 없으면 즉시 에러 (조용히 NaN 만들지 않기)
                missing_cols = set(cols) - set(df.columns)
                if missing_cols:
                    raise ValueError(f"DB 적재 대상 컬럼 누락: {sorted(missing_cols)}")

                sub = df[cols].astype(object).where(pd.notnull(df[cols]), None)
                data = [tuple(r) for r in sub.to_numpy()]

                for i in range(0, len(data), CHUNK_SIZE):
                    cur.executemany(sql, data[i:i+CHUNK_SIZE])
                    print(f"  inserted {min(i+CHUNK_SIZE, len(data))}/{len(data)}")

                conn.commit()
                ok += 1
                print(f"[DONE] {f.name} rows={len(df)}")

            except Exception as e:
                conn.rollback()
                fail += 1
                print(f"[FAIL] {f.name} -> {e}")

    conn.close()
    print("\n===================================")
    print(f"files ok={ok}, fail={fail}")
    print("ALL DONE")

# 실행
load_sales_files()


[START] seoul_sales_2023.csv
  inserted 20000/68643
  inserted 40000/68643
  inserted 60000/68643
  inserted 68643/68643
[DONE] seoul_sales_2023.csv rows=68643

[START] seoul_sales_2024.csv
  inserted 20000/67900
  inserted 40000/67900
  inserted 60000/67900
  inserted 67900/67900
[DONE] seoul_sales_2024.csv rows=67900

[START] seoul_sales_20251.csv
  inserted 16770/16770
[DONE] seoul_sales_20251.csv rows=16770

[START] seoul_sales_20252.csv
  inserted 16794/16794
[DONE] seoul_sales_20252.csv rows=16794

files ok=4, fail=0
ALL DONE
